# Preamble

WIP: reformating how I store anyons!

DONE.

FIX: eig -> eigh

TODO: 5- and 6-gon consistency.

In [1]:
import numpy as np
import itertools

Some Matrix functions:

In [2]:
def dag(M):
    return np.conjugate(M.T)
def matInv(M):
    return np.linalg.inv(M)
def is_eq(A,B):
    return np.array_equal(A,B)

In [3]:
A = np.array([[0, 1.j],[-1.j, 0]])
print(A)
print(is_eq(dag(A), matInv(A)))

[[ 0.+0.j  0.+1.j]
 [-0.-1.j  0.+0.j]]
True


# Fusion Rules and F- and R- Symbol Calculation

## Fusion

Using the Verlinde formula to get S-matrix and then using that to extract the fusion rules $N_{\mathcal{A}\mathcal{B}}^{\mathcal{C}}$:

$\mathcal{S}_{\mathcal{A}\mathcal{B}} = \frac{1}{|G|}\sum_{h_i^A \in C_A, h_j^B \in C_B, h^A_i h^B_j = h^B_j h^A_i} \chi^A((x_i^A)^{-1}h_j^B x_i^A)\chi^B((x_j^B)^{-1}h_i^A x_j^B)$,

where $x_i^A h_1^A(x_i^A)^{-1} = h_1^A$ which is a $C_A$ representative [alt. $q_c r \bar{q}_c = c$].

$N_{\mathcal{A}\mathcal{B}}^{\mathcal{C}} = \sum_\mathcal{L} \frac{\mathcal{S}_{\mathcal{A}\mathcal{L}}\mathcal{S}_{\mathcal{B}\mathcal{L}}\mathcal{S}_{\mathcal{C}\mathcal{L}}^*}{\mathcal{S}_{\mathcal{0}\mathcal{L}}}$.

## F- and R-

Imagine we hace a space $\mathcal{H} = \mathcal{H}_b \otimes \mathcal{H}_a$ we can project down to the space of irreducible representations by $\mathcal{P}_{c}^{ba}(\ket{c, \mu}) \propto \sum_{h \in G}\sum_{z \in Z(r)}R^\dag_{\mu\mu}(z) B_h^b A_{q_c z \bar{q}_c}^b \otimes B_{h^{-1}c}^a A_{q_c z \bar{q}_c}^a$, where $\ket{c, \mu} \in \mathcal{H}_c$.

This allows us to define the splitting tree map: $\phi_c^{ba}: \mathcal{H}_c \rightarrow \mathcal{H}_b \otimes \mathcal{H}_a$ as $\phi_c^{ba}(\ket{c, \mu}) = \text{eig}(\mathcal{P}_{c}^{ba}(\ket{c, \mu}), \lambda = 1) \times e^{i \phi(c, \mu)}$. The phase being set by requiring correct group action of $A_g^{ba} \circ \phi_c^{ba} \circ A_{\bar{g}}^{c} = \phi_c^{ba} $.

The braiding action $\mathcal{B}_{ab}: \mathcal{H}_a\otimes\mathcal{H}_b\rightarrow\mathcal{H}_b\otimes\mathcal{H}_a$ is given by: $Flip \circ (\sum_g A_g^a \otimes B_g^b)$.

Combining the two facts we can evaluate the R-symbol as: $\mathcal{R}_c^{ab} \propto (\phi_c^{ba}(\ket{c, \mu}))^\dag\mathcal{B}_{ab}(\phi_c^{ab}(\ket{c, \mu}))$, with condition $|\mathcal{R}_c^{ab}| = 1$.

### F- Symbol

In a similar manner we can calculate the F-symbol as : $(\mathcal{F}_a^{bcd})_{xy} \propto (\phi_y^{cd} \circ \phi_a^{by}(\ket{c,\mu}))^\dag\phi_x^{bc} \circ \phi_a^{xd}(\ket{c,\mu})$

## $D_4$

### Fusion Rule Calculations and Group Def

In [4]:
R = np.array([[0, -1],[1, 0]])
M = np.array([[1, 0],[0, -1]])
G = np.zeros([2,2,8])
G[:,:,0] = np.eye(2) # The D_4 group [via it's faithful 2d rep]
G[:,:,1] = R
G[:,:,2] = R@R
G[:,:,3] = R@R@R
G[:,:,4] = M
G[:,:,5] = M@R
G[:,:,6] = M@R@R
G[:,:,7] = M@R@R@R
g_lab = ['e','r','rr','rrr','m','mr','mrr','mrrr']
order = len(g_lab)

C_e = np.array([0])  # Conj Classes
C_r = np.array([1,3])
C_rr = np.array([2])
C_m = np.array([4,6])
C_mr = np.array([5,7])

Z_e = np.array([0, 1, 2, 3, 4, 5, 6, 7])  # Centers Classes
Z_r = np.array([0, 1, 2, 3])
Z_rr = np.array([0, 1, 2, 3, 4, 5, 6, 7])
Z_m = np.array([0, 2, 4, 6])
Z_mr = np.array([0, 2, 5, 7])

X_e = np.array([0]) # Coset Representatives
X_r = np.array([0,4])
X_rr = np.array([0])
X_m = np.array([0,3])
X_mr = np.array([0,3])


In [5]:
# Def the reps as Mats
def Triv(x):
    return np.array([[1]])
def A_r(x):
    if (x in C_mr) or (x in C_m):
        return np.array([[-1]])
    else:
        return np.array([[1]])
def A_mr(x):
    if (x in C_r) or (x in C_m):
        return np.array([[-1]])
    else:
        return np.array([[1]])
def A_m(x):
    if (x in C_mr) or (x in C_r):
        return np.array([[-1]])
    else:
        return np.array([[1]])
def E_D(x):
    return G[:,:,x] # Faithful 2d Rep
def B_i(x):
    if (x in [4,5,6,7]):
        return np.array([[-1]])
    elif (x in [0,2]):
        return np.array([[1]])
    else:
        return np.array([[0]])
def B_j(x):
    if (x in [2,6,7]):
        return np.array([[-1]])
    elif (x in [0,4,5]):
        return np.array([[1]])
    else:
        return np.array([[0]])
def B_k(x):
    if (x in [2,4,5]):
        return np.array([[-1]])
    elif (x in [0,6,7]):
        return np.array([[1]])
    else:
        return np.array([[0]])
def Om_1(x):
    if (x in [0,1,2,3]):
        return np.array([[(1.j**x)]])
    else:
        return np.array([[0]])
def Om_2(x):
    if (x in [0,1,2,3]):
        return np.array([[(1.j**(2*x))]])
    else:
        return np.array([[0]])
def Om_3(x):
    if (x in [0,1,2,3]):
        return np.array([[(1.j**(3*x))]])
    else:
        return np.array([[0]])

In [6]:
# Def the charges of reps
def triv(x):
    return 1
def a_r(x):
    if (x in C_mr) or (x in C_m):
        return -1
    else:
        return 1
def a_mr(x):
    if (x in C_r) or (x in C_m):
        return -1
    else:
        return 1
def a_m(x):
    if (x in C_mr) or (x in C_r):
        return -1
    else:
        return 1
def e_D(x):
    if (x in C_e):
        return 2
    elif (x in C_rr):
        return -2
    else:
        return 0
def b_i(x):
    if (x in [4,5,6,7]):
        return -1
    elif (x in [0,2]):
        return 1
    else:
        return 0
def b_j(x):
    if (x in [2,6,7]):
        return -1
    elif (x in [0,4,5]):
        return 1
    else:
        return 0
def b_k(x):
    if (x in [2,4,5]):
        return -1
    elif (x in [0,6,7]):
        return 1
    else:
        return 0
def om_1(x):
    if (x in [0,1,2,3]):
        return (1.j**x)
    else:
        return 0
def om_2(x):
    if (x in [0,1,2,3]):
        return (1.j**(2*x))
    else:
        return 0
def om_3(x):
    if (x in [0,1,2,3]):
        return (1.j**(3*x))
    else:
        return 0


In [7]:
# Def the anyons!
Z_s_D = [Z_e, Z_rr, Z_e, Z_e, Z_e, Z_rr, Z_rr, Z_rr, Z_e, Z_rr, Z_r, Z_r, Z_r, Z_r, Z_m, Z_m, Z_m, Z_m, Z_mr, Z_mr, Z_mr, Z_mr]
Dim_s_D = [1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
C_mods_D = [1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2]
chi_mods_D = [1,1,1,1,1,1,1,1,2,2,1,1,1,1,1,1,1,1,1,1,1,1]
Z_reps_D = [Triv, Triv, A_r, A_mr, A_m, A_r, A_mr, A_m, E_D, E_D, Triv, Om_1, Om_2, Om_3, Triv, B_i, B_j, B_k, Triv, B_i, B_j, B_k]
C_s_D = [C_e, C_rr, C_e, C_e, C_e, C_rr, C_rr, C_rr, C_e, C_rr, C_r, C_r, C_r, C_r, C_m, C_m, C_m, C_m, C_mr, C_mr, C_mr, C_mr]
Z_charges = [triv, triv, a_r, a_mr, a_m, a_r, a_mr, a_m, e_D, e_D, triv, om_1, om_2, om_3, triv, b_i, b_j, b_k, triv, b_i, b_j, b_k]
Particles_lab_D = ['O_e','O_rr', 'R_e', 'MR_e', 'M_e', 'R_rr', 'MR_rr', 'M_rr',\
     'E_e', 'E_rr',\
        'OM0_r', 'OM1_r', 'OM2_r', 'OM3_r', \
            'B1_m', 'B2_m', 'B3_m', 'B4_m', \
                'B1_mr', 'B2_mr', 'B3_mr', 'B4_mr']
N_anyons = 22
Particles_D = [(i, Dim_s_D[i], C_s_D[i],  C_mods_D[i], Z_s_D[i], Z_charges[i], Z_reps_D[i], chi_mods_D[i], Particles_lab_D[i]) for i in range(N_anyons)]

In [8]:
# Def some useful fuctions and particles [Many use G as a global variable!!!! order of cell exec is important]

def q(c):
    Q_C = [0, 0, 0, 4, 0, 0, 3, 3] # This for q_c instead
    return Q_C[c]


def grp(x):
    return G[:,:,x] # USE G AS A GLOBAL VAR!!! ---- CAREFUL !


def grp_i(i):
    return matInv(G[:,:,i])


def index(h):
    for i in range(8): # to be generalised
        if np.array_equal(h, G[:,:,i]):
            return i


def inv(x):
    for i in range(8): # to be generalised
        if np.array_equal(matInv(G[:,:,x]), G[:,:,i]):
            return i
# i, d, C, cd, Z, char, rep, repd, lab
S = np.zeros([22,22], complex)
for i, _, C_i, _, _, char_i, _, _,_ in Particles_D:
    for j, _, C_j, _, _, char_j, _, _,_ in Particles_D:
        s = 0 
        for i_class, elem_i_class in enumerate(C_i):
            for j_class, elem_j_class in enumerate(C_j):
                if is_eq(grp(elem_i_class)@grp(elem_j_class), grp(elem_j_class)@grp(elem_i_class)):
                    s+=char_i(index(grp_i(q(elem_i_class))@grp(elem_j_class)@grp(q(elem_i_class))))*char_j(index(grp_i(q(elem_j_class))@grp(elem_i_class)@grp(q(elem_j_class))))
        S[i,j] = s # maybe conj

S_D = S/8
# print('\n'.join([''.join(['{:4}'.format(item) for item in row]) 
#      for row in S]))
N_D = np.zeros([22,22,22])
for I in range(22):
    for J in range(22):
        for K in range(22):
            for L in range(22):
                N_D[I,J,K] += np.real(S_D[I,L]*S_D[J,L]*np.conj(S_D[K,L])/S_D[0,L])

Query S matrix:

In [9]:
an_1 = 10
an_2 = 15
print(Particles_lab_D[an_1],',', Particles_lab_D[an_2])
print('S_{aa}='+str(S_D[an_1, an_1]))
print('S_{bb}='+str(S_D[an_2, an_2]))
print('S_{ab}='+str(S_D[an_1, an_2]))
print('S_{ba}='+str(S_D[an_2, an_1]))
print('S_{0a}='+str(S_D[0, an_2]))
print('S_{a0}='+str(S_D[an_2, 0]))
print('S_{0a}='+str(S_D[0, an_1]))
print('S_{a0}='+str(S_D[an_1, 0]))

OM0_r , B2_m
S_{aa}=(0.5+0j)
S_{bb}=(0.5+0j)
S_{ab}=0j
S_{ba}=0j
S_{0a}=(0.25+0j)
S_{a0}=(0.25+0j)
S_{0a}=(0.25+0j)
S_{a0}=(0.25+0j)


Query Fusion rules:

In [10]:
i_1 = 10 # Prints fusion rules
i_2 = 10
print(Particles_lab_D[i_1],'x', Particles_lab_D[i_2], '=')
for i, indicator in enumerate(N_D[i_1,i_2,:]):
    if indicator != 0:
        print(int(indicator),'x', Particles_lab_D[i])
S_r = 8*np.real(S)

OM0_r x OM0_r =
1 x O_e
1 x O_rr
1 x R_e
1 x R_rr


### Defining Representation Matrices of $D(D_4)$

#### $A_g$ :

In [11]:
# i, d, C, cd, Z, char, rep, repd, lab
def a_mat(an_type, g):
    # Basis is |c \mu> 
    (id, d, C, C_mod, Z_set, char, Z_rep, chi_mod, lab) = Particles_D[an_type]
    # use q(c) instead of X set
    A = np.zeros([d, d], dtype=complex)
    for i_c, c in enumerate(C):
        for mu in range(chi_mod):
            for i_cp, c_prim in enumerate(C):
                for nu in range(chi_mod):
                    if is_eq(grp(c_prim), grp(g)@grp(c)@grp_i(g)):
                        rep_map = Z_rep(index( grp_i(q(c_prim))@grp(g)@grp(q(c)) ))
                        mat_elem = rep_map[mu,nu]
                        A[i_cp*chi_mod+nu, i_c*chi_mod+mu]=mat_elem
    return A

#### $B_h:$

In [12]:
def b_mat(an_type, g):
    # Basis is |c \mu> 
    (id, d, C, C_mod, Z_set, char, Z_rep, chi_mod, lab) = Particles_D[an_type]
    # use q(c) instead of X set
    B = np.zeros([d, d], dtype=complex)
    for i_c, c in enumerate(C):
        for mu in range(chi_mod):        
            if g == c:
                B[i_c*chi_mod+mu, i_c*chi_mod+mu]=1
    return B

#### Query the Matrices:

In [13]:
an = 11
g = 1
print(Particles_lab_D[an])
print(g_lab[g])
print(a_mat(an, g))
print(b_mat(an, g))

OM1_r
r
[[ 0.+1.j  0.+0.j]
 [ 0.+0.j -0.-1.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


### The fusion projectors and plitting tree maps

In [14]:
def fusion_products(an_1, an_2):
    res = []
    for i, indicator in enumerate(N_D[an_1,an_2,:]):
        if indicator != 0:
            res += [i]
    return res

def do_they_fuse(an_1, an_2, an_3):
    return an_3 in fusion_products(an_1, an_2)

def proj_norm(P):
    a = np.trace(P)
    b = np.trace(P@P)
    return P*a/b

def splitting_tree(an_1, an_2, an_3, id): #Only good if id = 0
    Pro = rep_proj_spec(an_1, an_2, an_3, id)
    w, v = np.linalg.eigh(Pro)
    for i_lam, lam in enumerate(w):
        if lam == 1:
            Vec = v[:,i_lam]
    return Vec

def splitting_tree_proper(an_1, an_2, an_3): #Returns a properlt gauge fixed basis
    (_, d_3, _, _, _, _, _, _, _) = Particles_D[an_3]
    (_, d_2, _, _, _, _, _, _, _) = Particles_D[an_2]
    (_, d_1, _, _, _, _, _, _, _) = Particles_D[an_1]
    Basis_unfixed = []
    order = np.size(G[0,0,:])
    for i_3 in range(d_3):
        Basis_unfixed += [splitting_tree(an_1, an_2, an_3, i_3)]
    Basis_fixed = Basis_unfixed
    if d_3 == 1:
        return Basis_fixed
    else:
        for i_3 in range(1,d_3):
            not_done = 1
            for g in range(order):
                if not_done:
                    Vec_3 = np.zeros([d_1*d_2], dtype=complex)
                    for mat_mul in range(d_3):
                        Vec_3 += a_mat(an_3, g)[0,mat_mul]*Basis_unfixed[mat_mul]
                    Vec_12 = np.kron(a_mat(an_1, g), a_mat(an_2, g))@Basis_unfixed[0]
                    overlap_12 = np.vdot(Basis_unfixed[i_3], Vec_3)
                    overlap_3 = np.vdot(Basis_unfixed[i_3], Vec_12)
                    if overlap_12 != 0 and overlap_3 != 0:
                        Basis_fixed[i_3] = Basis_unfixed[i_3]*overlap_12/overlap_3
                        not_done = 0
    return Basis_fixed




def rep_proj(an_1, an_2, an_3):
    (id_3, _, C_3, _, Z_3, char_3, Z_rep_3, chi_mod_3, _) = Particles_D[an_3]
    (_, d_2, _, _, _, _, _, _, _) = Particles_D[an_2]
    (_, d_1, _, _, _, _, _, _, _) = Particles_D[an_1]
    Res = np.zeros([d_1*d_2,d_1*d_2], dtype=complex)
    for c in C_3:
        for z in Z_3:
            for h in range(8):
                Res += np.conjugate(char_3(z))*np.kron( b_mat(an_1, index( grp_i(h)@grp(c) ) ) @ a_mat(an_1,index( grp(q(c))@grp(z)@grp_i(q(c)) )),
                b_mat(an_2,h) @ a_mat(an_2,index( grp(q(c))@grp(z)@grp_i(q(c)) )) )
    return proj_norm(Res) # Normalisation issues :(

def rep_proj_spec(an_1, an_2, an_3, id):# Basis is |c \mu> # NOT SURE if the FORM is correct 
    (id_3, _, C_3, _, Z_3, char_3, Z_rep_3, chi_mod_3, _) = Particles_D[an_3]
    (_, d_2, _, _, _, _, _, _, _) = Particles_D[an_2]
    (_, d_1, _, _, _, _, _, _, _) = Particles_D[an_1]
    Res = np.zeros([d_1*d_2,d_1*d_2], dtype=complex)
    i_c = int(np.floor(id/chi_mod_3))
    c = C_3[i_c]
    mu = np.mod(id,chi_mod_3)
    for z in Z_3:
        for h in range(8):
            Res += Z_rep_3(z)[mu,mu].conj()*np.kron( b_mat(an_1, # Investigate does this one work???
            index( grp_i(h)@grp(c) ) ) @ a_mat(an_1,
            index( grp(q(c))@grp(z)@grp_i(q(c)) )),
            b_mat(an_2,h) @ a_mat(an_2,index( grp(q(c))@grp(z)@grp_i(q(c)) )) )
    return proj_norm(Res) # Normalisation issues :(



In [15]:
np.size(G[0,0,:])

8

### The R-sym Calculation:

In [16]:
def r_symbol(an_1, an_2, an_3):
    (_, d_2, _, _, _, _, _, _, _) = Particles_D[an_2]
    (_, d_1, _, _, _, _, _, _, _) = Particles_D[an_1]
    T = np.zeros([d_1*d_2, d_1*d_2], dtype=complex)
    # T_k = np.zeros([d_1*d_2, d_1*d_2], dtype=complex)
    for g in range(order):
        T += np.kron(a_mat(an_1,g), b_mat(an_2,g))
        # T_k += np.kron(b_mat(an_2,g), a_mat(an_1,g))
    # print('apply', T)
    F = np.zeros([d_1*d_2, d_1*d_2])
    for i in range(d_1):
        for j in range(d_2):
            for m in range(d_2):
                for n in range(d_1):
                    if i == n and j == m:
                        F[m*d_1+n, i*d_2 + j] = 1

    # Contrcut the Splitting Tree Basis!
    Vec_1 = splitting_tree(an_2, an_1, an_3,0)
    Vec_2 = splitting_tree(an_1, an_2, an_3,0) 
    Vec_2 = F@T@Vec_2 # The other way gives 0
    # print(Vec_1)
    # print(Vec_2)
    overlap = np.round(np.vdot(Vec_1, Vec_2),2) # issue !!!!
    # print(Vec_1, Vec_2)
    return overlap
    
def fusion_and_braid(an_1, an_2): # Depends on a lot of outside variables!!!
    # print(Particles_lab_Q[an_1],'x', Particles_lab_Q[an_2], '=')
    # Reduce text
    pr = []
    for i, indicator in enumerate(N_D[an_1,an_2,:]):
        if np.round(indicator,1) != 0:
            # Calculate the R-Symbol
            #print(int(indicator),'x', Particles_lab_Q[i], 'with R_symb', str(r_symbol(an_1,an_2, i)))
            pr += [(i, r_symbol(an_1,an_2, i))]
    print(an_1, 'x', an_2, '=', pr)
    return 0

The Results, quesries of fusion and R-symbols:

In [17]:
fusion_and_braid(21,21)

21 x 21 = [(0, (-1+0j)), (3, (-1+0j)), (5, (-1+0j)), (7, (1+0j))]


0

In [18]:
# for an_1, x, y, z in Particles_D:
#     for an_2, xx, yy, zz in Particles_D:
#         fusion_and_braid(an_1, an_2)

### F- Symbols 


In [19]:
print(fusion_products(10,10))
print(do_they_fuse(10,10,4))
np.zeros([2])

[0, 1, 2, 5]
False


array([0., 0.])

In [20]:
def f_mat(an_a, an_b, an_c, an_d):
    (_, d_a, _, _, _, _, _, _, _) = Particles_D[an_a]
    (_, d_b, _, _, _, _, _, _, _) = Particles_D[an_b]
    (_, d_c, _, _, _, _, _, _, _) = Particles_D[an_c]
    (_, d_d, _, _, _, _, _, _, _) = Particles_D[an_d]
    x_ids = []
    y_ids = []
    print('x=')
    for x in fusion_products(an_b, an_c):
        if do_they_fuse(x,an_d,an_a):
            x_ids+=[x]
            print(Particles_lab_D[x])
    print('y=')
    for y in fusion_products(an_c, an_d):
        if do_they_fuse(an_b,y,an_a):
            y_ids+=[y]
            print(Particles_lab_D[y])
    F = np.zeros([len(x_ids), len(y_ids)], dtype=complex)

    for i_x, x in enumerate(x_ids):
        for i_y, y in enumerate(y_ids):
            (_, d_x, _, _, _, _, _, _, _) = Particles_D[x]
            (_, d_y, _, _, _, _, _, _, _) = Particles_D[y]
            i_vec_a = 0
            vec_xd = splitting_tree(x, an_d, an_a, i_vec_a)
            vec_by = splitting_tree(an_b, y, an_a, i_vec_a)
            vec_bcd_via_x = np.zeros([d_a*d_b*d_c], dtype=complex)
            vec_bcd_via_y = np.zeros([d_a*d_b*d_c], dtype=complex)
            x_basis = splitting_tree_proper(an_b, an_c, x)
            y_basis = splitting_tree_proper(an_c, an_d, y)
            for id_x in range(d_x):
                for id_d in range(d_d):
                    e_d = np.zeros([d_d])
                    e_d[id_d] = 1
                    vec_bcd_via_x += vec_xd[id_x*d_d + id_d]*np.kron(x_basis[id_x] ,e_d)
            for id_y in range(d_y):
                for id_b in range(d_b):
                    e_b = np.zeros([d_b])
                    e_b[id_b] = 1
                    vec_bcd_via_y += vec_by[id_b*d_y + id_y]*np.kron(e_b,y_basis[id_y])
            F[i_x, i_y] += np.vdot(vec_bcd_via_y, vec_bcd_via_x)
    F = np.round(F, 2)
    print('F=',F)
    return F

    

In [21]:
def f_mat_elem(an_a, an_b, an_c, an_d, x, y, i_vec_a): # use to debug
    (_, d_a, _, _, _, _, _, _, _) = Particles_D[an_a]
    (_, d_b, _, _, _, _, _, _, _) = Particles_D[an_b]
    (_, d_c, _, _, _, _, _, _, _) = Particles_D[an_c]
    (_, d_d, _, _, _, _, _, _, _) = Particles_D[an_d]
    F = 0.j
    (_, d_x, _, _, _, _, _, _, _) = Particles_D[x]
    (_, d_y, _, _, _, _, _, _, _) = Particles_D[y]
    vec_xd = splitting_tree(x, an_d, an_a, i_vec_a)
    vec_by = splitting_tree(an_b, y, an_a, i_vec_a)
    vec_bcd_via_x = np.zeros([d_a*d_b*d_c], dtype=complex)
    vec_bcd_via_y = np.zeros([d_a*d_b*d_c], dtype=complex)
    x_basis = splitting_tree_proper(an_b, an_c, x)
    y_basis = splitting_tree_proper(an_c, an_d, y)
    for id_x in range(d_x):
        for id_d in range(d_d):
            e_d = np.zeros([d_d])
            e_d[id_d] = 1
            vec_bcd_via_x += vec_xd[id_x*d_d + id_d]*np.kron(x_basis[id_x] ,e_d)
    for id_y in range(d_y):
        for id_b in range(d_b):
            e_b = np.zeros([d_b])
            e_b[id_b] = 1
            vec_bcd_via_y += vec_by[id_b*d_y + id_y]*np.kron(e_b, y_basis[id_y])
    print('x up', np.round(np.real(vec_bcd_via_x),2))
    print('y down',np.round(np.real(vec_bcd_via_y), 2))
    F += np.vdot(vec_bcd_via_x, vec_bcd_via_y)
    F = np.round(F, 2)
    return F

    

Query the F-matrices

In [22]:
F = f_mat(14, 10, 14, 10)

x=
B1_mr
B2_mr
y=
B1_mr
B2_mr
F= [[ 1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]


In [23]:
np.round(F@dag(F), 2)

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])